In [385]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request as urllib2
from io import StringIO
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import hgutils
import time
stopwatch = hgutils.timer("ALM Algo")
import threading
import concurrent.futures
import fastparquet
import pyarrow
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"

import warnings
warnings.filterwarnings('ignore')

In [386]:
goal_amount = 1_00_000
startDate = dt.date(2015,1,1)
goal_length = ['1m','2m','3m','6m','1y','2y','3y','5y']

# collection_window = ['1d','3d','7d','1m','2m','3m','6m','1y']
collection_window = ['1d']

In [387]:
def emi_days_range(emi_count):
    return [i*30 for i in range(emi_count)]

def date_range(start_date,end_date,frequency=1):
    current_date = start_date
    yield current_date
    current_date = current_date + dt.timedelta(days=frequency)
    while current_date<=end_date:
        yield current_date
        current_date = current_date + dt.timedelta(days=frequency)

def generate_amfi_url (date,base_url = 'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?frmdt='):
    date_str = date.strftime("%d-%b-%Y")
    return base_url+date_str

def get_day_df_recursive (url, break_minutes = 5, selected_schemes = None):
    try:
        page = urllib2.urlopen(url)
        soup = BeautifulSoup(page.read())
        df = pd.read_csv(StringIO(soup.get_text()),sep=';')

        df['Scheme'] = df['Scheme Code']
        df['AMC'] = df['Scheme Code']

        df.loc[~df['Scheme Code'].str.contains("Schemes"),"Scheme"] = None
        df.loc[~df['Net Asset Value'].isnull(),"AMC"] = None

        df[['Scheme','AMC']] = df[['Scheme','AMC']].fillna(method='ffill')

        df['Scheme Type'] = df['Scheme'].str.split(pat="(",expand=True)[1].str.replace(")","").str.strip()
        df['Scheme'] = df['Scheme'].str.split(pat="(",expand=True)[0].str.strip()   

        df.dropna(subset='Scheme Name',inplace=True)

        if selected_schemes is not None:
            df = df[df['Scheme Name'].isin(selected_schemes)]

        
    except:
        stopwatch.start("Break Time")
        time.sleep(break_minutes*60)
        df = get_day_df_recursive(url,break_minutes)
        stopwatch.stop()
    return df

def download_daily_amfi_data(date,selected_schemes=None):
    df = get_day_df_recursive(generate_amfi_url(date),break_minutes=5,selected_schemes=required_schemes)
    df.astype(str).to_parquet("amfi_data_hdfc/"+date.strftime("%Y%b%d")+'.parquet')
    if date.day == 1:
        print(date.strftime("%Y%b%d"))

In [388]:
cpi = pd.read_excel("cpi.xlsx")
cpi['date'] = pd.to_datetime(cpi['Period'])
cpi['Period'] = cpi['date'].dt.strftime("%Y-%m")

In [389]:
base = dt.date.today()
base = dt.datetime(base.year,base.month,base.day)

date_list = list(pd.date_range(startDate,end=dt.date.today()).date)

df = pd.DataFrame({'startDate':[date_list],'length':[goal_length],'collection_window':[collection_window]})
df = (df
    .explode('startDate')
    .explode('length')
    .explode('collection_window')
    .sort_values(by = 'startDate'))
# df['date'] = df['date'].dt.date
df['length_days'] = df['length'].str[:-1].astype(int) * df['length'].str[-1:].map({'d':1,'m':30,'y':360})
df['endDate'] = df['startDate'] + pd.to_timedelta(df['length_days'], unit='D')

df = df[df['endDate']<=dt.date.today()]

df['collection_days'] = df['collection_window'].str[:-1].astype(int) * df['collection_window'].str[-1:].map({'d':1,'m':30,'y':360})

df['emi_count'] = (df['length_days']/30).astype(int)

df['goal_startDate'] = goal_amount

df['startDate_period'] = df['startDate'].apply(lambda x: x.strftime("%Y-%m"))
df['endDate_period'] = df['endDate'].apply(lambda x: x.strftime("%Y-%m"))

df = (df
    .merge(cpi[['Period','Index']],left_on='startDate_period',right_on='Period')
    .drop(columns='Period')
    .rename(columns={'Index':'startDate_index'})
    .merge(cpi[['Period','Index']],left_on='endDate_period',right_on='Period')
    .drop(columns='Period')
    .rename(columns={'Index':'endDate_index'}))

df['goal_endDate'] = df['endDate_index'] * df['goal_startDate'] / df['startDate_index']

df['emi'] = df['goal_startDate'] / df['emi_count']

df['emi_days_range'] = df['emi_count'].apply(emi_days_range)
df = (df.explode('emi_days_range'))
df['date'] = df['startDate'] + pd.to_timedelta(df['emi_days_range'], unit='D')



df['days_to_maturity'] = (df['endDate'] - df['date']).dt.days

df_required_cols = ['startDate', 'length', 'collection_window','endDate',
       'goal_endDate','emi', 'emi_days_range', 'date', 'days_to_maturity']

df = df[df_required_cols]

df['is_investment'] = True
df.loc[df['days_to_maturity']==0,'is_investment'] = False

df.head(2)

,startDate,length,collection_window,endDate,goal_endDate,emi,emi_days_range,date,days_to_maturity,is_investment
0,2015-01-01,1m,1d,2015-01-31,100000.000000,100000.0,0,2015-01-01,30,True
1,2015-01-01,2m,1d,2015-03-02,100506.329114,50000.0,0,2015-01-01,60,True


In [396]:
read_parquet = False
update_csv = True

if read_parquet:
    stopwatch.start("Reading parquet files")
    fund = pd.read_parquet('amfi_data/',use_nullable_dtypes=True, engine='pyarrow')
    stopwatch.stop(print=True, verbose=0)

    stopwatch.start("Only required schemes")
    scheme = pd.read_excel("schemes.xlsx",sheet_name='Use')
    # required_schemes = list(scheme['Scheme Name'].unique())
    # fund = fund[fund['Scheme Name'].isin(required_schemes)]
    stopwatch.stop()

    stopwatch.start("Saving parquet of required funds")
    fund = (fund
        .merge(scheme[['Scheme Name','Duration Days']],
            on='Scheme Name')
        )
    fund.to_csv("Funds.csv",index=False)
    stopwatch.stop()

if (not read_parquet) & (update_csv):
    stopwatch.start("Updating Schemes Data")
    scheme = pd.read_excel("schemes.xlsx",sheet_name='Use')
    fund = pd.read_csv("Fund.csv")
    fund = (fund
        .drop(columns=['Duration Days'])
        .merge(scheme[['Scheme Name','Duration Days']],
            on='Scheme Name')
        )
    stopwatch.stop()

if read_parquet | update_csv:
    stopwatch.start("Saving Fund")
    fund = fund[['Scheme Code',
        'Scheme Name','Net Asset Value',
        'Date','Duration Days']]
    fund.to_csv("Fund.csv",index=False)
    stopwatch.stop()


stopwatch.start("Preparing Funds df")

fund['date'] = pd.to_datetime(fund['Date']).dt.date
fund.drop(columns=['Date'],inplace=True)
expanded_funds = fund[['Scheme Code','Scheme Name','Duration Days']].drop_duplicates()
expanded_funds['date'] = [list(pd.date_range(dt.date(2010,1,1),end=dt.date.today()).date) for _ in range(expanded_funds.shape[0])]
fund = (expanded_funds
    .explode('date')
    .merge(fund,on=['Scheme Code','Scheme Name','Duration Days','date'],how='left')
    .sort_values(by=['Scheme Name','date']))

fund['Net Asset Value'] = (fund['Net Asset Value']
        .fillna(method='ffill')
        .astype(float))

fund['Scheme Code'] = fund['Scheme Code'].astype(str)

stopwatch.stop()

In [391]:
min_days_to_maturity = df[df['is_investment']]['days_to_maturity'].min()
max_days_to_maturity = df[df['is_investment']]['days_to_maturity'].max()
available_durations = list(fund['Duration Days'].unique())
available_durations.sort()

duration_mapping = {0:0}
current_pos = 1
for d in range(min_days_to_maturity,max_days_to_maturity+1):
    if d >= available_durations[current_pos]:
        if current_pos < len(available_durations)-1:
            current_pos += 1
    duration_mapping[d] = int(available_durations[current_pos-1])

df['matched_duration'] = (df['days_to_maturity']
                    .map(duration_mapping)
                    .astype(int))


In [392]:
df = df.merge(fund,left_on=['date','matched_duration'], right_on=['date','Duration Days'],how='left')
df['Units Bought'] = df['emi'] / df['Net Asset Value']
df = df.merge(fund[['Scheme Code','date','Net Asset Value']],
    left_on=['endDate','Scheme Code'],right_on=['date','Scheme Code'],suffixes=("","_endDate"))

df['value_endDate'] = df['Units Bought'] * df['Net Asset Value_endDate']



In [393]:
df.groupby(['startDate','length'],as_index=False).agg({'goal_endDate':'mean','value_endDate':'sum'})

,startDate,length,goal_endDate,value_endDate
0,2015-01-01,1m,100000.000000,1.006678e+05
1,2015-01-01,1y,104641.350211,7.033079e+06
2,2015-01-01,2m,100506.329114,1.010525e+05
3,2015-01-01,2y,107679.324895,1.098736e+05
4,2015-01-01,3m,101012.658228,1.014105e+05
...,...,...,...,...
18075,2022-07-28,1m,100464.306442,1.004556e+05
18076,2022-07-29,1m,100464.306442,1.004555e+05
18077,2022-07-30,1m,100464.306442,1.004563e+05
18078,2022-07-31,1m,100464.306442,1.004532e+05


In [394]:
df[(df['startDate']==dt.date(2015,1,1)) & (df['length']=='1y')].to_excel('sample.xlsx',index=False)

In [398]:
fund[(fund['Scheme Code']=='119091')&(fund['date']>=dt.date(2015,8,20))]

,Scheme Code,Scheme Name,Duration Days,date,Net Asset Value
6731,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2015-08-20,28.5123
6732,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2015-08-21,28.5184
6733,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2015-08-22,28.5184
6734,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2015-08-23,28.5306
6735,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2015-08-24,28.5365
...,...,...,...,...,...
9343,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2022-10-14,4286.8496
9344,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2022-10-15,4286.8496
9345,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2022-10-16,4286.8496
9346,119091,HDFC Liquid Fund - Growth Option - Direct Plan,30,2022-10-17,4286.8496
